In [10]:
from py2neo import Graph
from community import best_partition

# 连接到Neo4j数据库
graph = Graph("bolt://198.18.31.159:7687", auth=("neo4j", "123456"))

# 执行Cypher查询以获取图的边和节点信息
cypher_query = """
MATCH (p:Process)-[r:create_process]->(c:Process)
RETURN id(p) as source, id(c) as target, r.time as timestamp, r.uid as uid_weight
"""
result = graph.run(cypher_query)

# 归一化时间戳
timestamps = [record["timestamp"] for record in result if record["timestamp"] is not None]


if timestamps:
    max_timestamp = max(timestamps)
    min_timestamp = min(timestamps)

    edges = [
        (
            record["source"],
            record["target"],
            {
                "weight": (record["timestamp"] - min_timestamp) / (max_timestamp - min_timestamp) + int(record["uid_weight"])
            },
        )
        for record in result if record["timestamp"] is not None
    ]

    print(edges)
    # 使用community库中的Louvain算法执行社区检测
    partition = best_partition(edges, weight="weight")

    # 输出社区信息
    print(partition)
else:
    print("No valid timestamps found.")


[]


AttributeError: 'list' object has no attribute 'is_directed'